In [31]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [32]:
out='../output/'
inb='../input/flows/'
pref='fpw'
inp = inb + pref + '/'

In [33]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-04-04,4,BOT,0,ZIMA
1,2016-04-04,16,MOZN_EDM,0,ZIMA
2,2016-04-04,16,MOZN_HCB,0,MOZN_EDM
3,2016-04-04,4,RSAN,0,BOT
4,2016-04-04,18,ZAMZ,0,ZAMC
...,...,...,...,...,...
35995,2023-04-30,0,ZIM,23,ZIMA
35996,2023-04-30,0,ZIMA,23,BOT
35997,2023-04-30,0,ZIMA,23,ZIM
35998,2023-04-30,0,ZIMA,23,MOZN_EDM


In [34]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

C:\Users\kabom\AppData\Local\Temp\ipykernel_29868\1849715270.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dam=dam.drop(['date'],1)


price  volume  year  month  day  day_of_week   value  \
date       hour                                                          
2017-03-06 4      29.00    50.0  2017      3    6            0  1450.0   
           5      29.00    50.0  2017      3    6            0  1450.0   
           6      65.00     0.0  2017      3    6            0     0.0   
           7      97.21     0.0  2017      3    6            0     0.0   
           8      97.21     0.0  2017      3    6            0     0.0   
...                 ...     ...   ...    ...  ...          ...     ...   
2023-05-31 19    160.00     0.0  2023      5   31            2     0.0   
           20     80.00     0.0  2023      5   31            2     0.0   
           21     80.00     0.0  2023      5   31            2     0.0   
           22     52.50    40.0  2023      5   31            2  2100.0   
           23     52.50    40.0  2023      5   31            2  2100.0   

                      cat  
date       hour            
2017-03-06 4          off  
           5     standard  
           6      morning  
           7      morning  
           8      morning  
...                   ...  
2023-05-31 19     evening  
           20         off  
           21         off  
           22         off  
           23         off  

[50800 rows x 8 columns]

In [35]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

C:\Users\kabom\AppData\Local\Temp\ipykernel_29868\2288514685.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  flow=flow.dropna(0)


,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2017-03-06,4,20,BOT,ZIMA,2017,3,6,0,29.0,580.0
1,2017-03-06,4,20,RSAN,BOT,2017,3,6,0,29.0,580.0
2,2017-03-06,4,30,ZAMZ,ZAMC,2017,3,6,0,29.0,870.0
3,2017-03-06,4,20,ZIM,ZAMZ,2017,3,6,0,29.0,580.0
4,2017-03-06,4,20,ZIMA,ZIM,2017,3,6,0,29.0,580.0
...,...,...,...,...,...,...,...,...,...,...,...
188791,2023-04-30,23,0,ZIM,ZIMA,2023,4,30,6,65.0,0.0
188792,2023-04-30,23,0,ZIMA,BOT,2023,4,30,6,65.0,0.0
188793,2023-04-30,23,0,ZIMA,ZIM,2023,4,30,6,65.0,0.0
188794,2023-04-30,23,0,ZIMA,MOZN_EDM,2023,4,30,6,65.0,0.0


In [36]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2017-03-06,4,20,580.0,29.0,ZIMA,2017,3,6,0
1,2017-03-06,4,20,580.0,29.0,BOT,2017,3,6,0
2,2017-03-06,4,30,870.0,29.0,ZAMC,2017,3,6,0
3,2017-03-06,4,20,580.0,29.0,ZAMZ,2017,3,6,0
4,2017-03-06,4,20,580.0,29.0,ZIM,2017,3,6,0
...,...,...,...,...,...,...,...,...,...,...
188791,2023-04-30,23,0,-0.0,65.0,ZIM,2023,4,30,6
188792,2023-04-30,23,0,-0.0,65.0,ZIMA,2023,4,30,6
188793,2023-04-30,23,0,-0.0,65.0,ZIMA,2023,4,30,6
188794,2023-04-30,23,0,-0.0,65.0,ZIMA,2023,4,30,6


In [37]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [38]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week   index  flow   value  \
date       hour                                                              
2017-03-06 4      RSA  2017      3    6            0       1   -20  -580.0   
           4     ZAMC  2017      3    6            0       2    30   870.0   
           4     ZAMZ  2017      3    6            0       5   -10  -290.0   
           5      RSA  2017      3    6            0       6   -20  -580.0   
           5     ZAMC  2017      3    6            0       7    30   870.0   
...               ...   ...    ...  ...          ...     ...   ...     ...   
2021-10-31 21    ZAMC  2021     10   31            6  118981    20  1001.4   
           22     MOZ  2021     10   31            6  118985   -20 -1001.4   
           22    ZAMC  2021     10   31            6  118987    20  1001.4   
           23     MOZ  2021     10   31            6  118991   -20 -1001.4   
           23    ZAMC  2021     10   31            6  118993    20  1001.4   

                 price       cat  
date       hour                   
2017-03-06 4     29.00       off  
           4     29.00       off  
           4     58.00       off  
           5     29.00  standard  
           5     29.00  standard  
...                ...       ...  
2021-10-31 21    50.07       off  
           22    50.07       off  
           22    50.07       off  
           23    50.07       off  
           23    50.07       off  

[49182 rows x 10 columns]

In [39]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum     value  hour_count  \
year month node cat      side                                      
2017 3     RSA  evening  supply       -200  -17224.0          10   
                morning  consume        50    1748.0           5   
                         supply       -200  -17224.0          10   
                off      consume      1500   36256.0          45   
                         supply      -2880  -83778.4         144   
...                                    ...       ...         ...   
2021 9     ZAMC standard consume       440   19662.4          22   
     10    MOZ  off      supply      -6320 -302278.4         316   
                standard supply       -420  -20392.0          21   
           ZAMC off      consume      6320  302278.4         316   
                standard consume       420   20392.0          21   

                                  price_weight_mean  price_mean      power  
year month node cat      side                                               
2017 3     RSA  evening  supply           86.120000   86.120000 -20.000000  
                morning  consume          34.960000   34.960000  10.000000  
                         supply           86.120000   86.120000 -20.000000  
                off      consume          24.170667   25.968889  33.333333  
                         supply           29.089722   29.089722 -20.000000  
...                                             ...         ...        ...  
2021 9     ZAMC standard consume          44.687273   44.687273  20.000000  
     10    MOZ  off      supply           47.828861   47.828861 -20.000000  
                standard supply           48.552381   48.552381 -20.000000  
           ZAMC off      consume          47.828861   47.828861  20.000000  
                standard consume          48.552381   48.552381  20.000000  

[572 rows x 6 columns]

In [40]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')